# Import libraries

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [3]:
# Import file
data = pd.read_csv('medical_examination.csv', index_col='id')
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## Task 1

Create a chart similar to examples/Figure_1.png, where we show the counts of good and bad outcomes for the cholesterol, gluc, alco, active, and smoke variables for patients with cardio=1 and cardio=0 in different panels.

In [62]:
def calc_overweight(dataframe):
    '''
    A simple function to caclulate BMI
    '''
    dataframe['overweight'] = (dataframe['weight'] / ((dataframe['height']/100) **2)).to_numpy()
    dataframe['overweight'] = np.where(dataframe['overweight'] < 25, 0, 1)

    # Normalize the data by making 0 always good and 1 always bad. If the value of cholesterol         or gluc is 1, make the value 0. If the value is more than 1, make the value 1.
    dataframe['cholesterol'] = np.where(dataframe['cholesterol'] <= 1, 0, 1)
    dataframe['gluc'] = np.where(dataframe['gluc'] <= 1, 0, 1)
    return dataframe


In [63]:
test = calc_overweight(data)
test.head(10)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,overweight
id,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,0,0,0,0,1,0,0
1,20228,1,156,85.0,140,90,1,0,0,0,1,1,1
2,18857,1,165,64.0,130,70,1,0,0,0,0,1,0
3,17623,2,169,82.0,150,100,0,0,0,0,1,1,1
4,17474,1,156,56.0,100,60,0,0,0,0,0,0,0
8,21914,1,151,67.0,120,80,1,1,0,0,0,0,1
9,22113,1,157,93.0,130,80,1,0,0,0,1,0,1
12,22584,2,178,95.0,130,90,1,1,0,0,1,1,1
13,17668,1,158,71.0,110,70,0,0,0,0,1,0,1


## Create dataframe for cardio_0 and 1

In [70]:
cardio_0 = data[data['cardio'] == 0].iloc[:,-7:].set_index('cardio')
cardio_0.head()

,cholesterol,gluc,smoke,alco,active,overweight
cardio,,,,,,
0,0,0,0,0,1,0
0,0,0,0,0,0,0
0,1,1,0,0,0,1
0,1,0,0,0,1,1
0,0,0,0,0,1,1


In [71]:
cardio_1 = data[data['cardio'] == 1].iloc[:,-7:].set_index('cardio')
cardio_1.head()

,cholesterol,gluc,smoke,alco,active,overweight
cardio,,,,,,
1,1,0,0,0,1,1
1,1,0,0,0,0,0
1,0,0,0,0,1,1
1,1,1,0,0,1,1
1,0,0,0,0,0,1


In [61]:
# put the needed columns to a list
plt_columns = data.iloc[:,-7:].columns.sort_values().to_list()
plt_columns

['active', 'alco', 'cardio', 'cholesterol', 'gluc', 'overweight', 'smoke']